In [1]:
# ==================== 单元2：导入库 ====================
import requests
import json
import time
import base64
from PIL import Image
import io
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import asyncio
from IPython.display import Audio, display, HTML
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 库导入完成")

✅ 库导入完成


In [2]:
# ==================== 单元3：全局配置 ====================
BASE_URL = "http://localhost:8000"
USER_ID = "test_user_001"
SESSION_ID = f"session_{int(time.time())}"
NAV_SESSION_ID = None

# 测试数据存储
TEST_RESULTS = {
    'health_check': [],
    'voice_text': [],
    'perception': [],
    'navigation': [],
    'websocket': [],
    'performance': [],
    'errors': []
}

print(f"✅ 配置完成")
print(f"📍 BASE_URL: {BASE_URL}")
print(f"👤 USER_ID: {USER_ID}")
print(f"🔑 SESSION_ID: {SESSION_ID}")

✅ 配置完成
📍 BASE_URL: http://localhost:8000
👤 USER_ID: test_user_001
🔑 SESSION_ID: session_1765980687


In [22]:
# ==================== 单元4：完整健康检查 ====================
print("=" * 70)
print("🏥 系统健康检查")
print("=" * 70)

checks = []

# 1. 服务可达性
try:
    response = requests.get(f"{BASE_URL}/", timeout=5)
    checks.append({
        '检查项': '服务可达性',
        '状态': '✅ 通过' if response.status_code == 200 else '❌ 失败',
        '详情': f"状态码: {response.status_code}"
    })
    root_data = response.json()
    print(f"服务名称: {root_data.get('service', 'N/A')}")
    print(f"版本: {root_data.get('version', 'N/A')}")
except Exception as e:
    checks.append({
        '检查项': '服务可达性',
        '状态': '❌ 失败',
        '详情': str(e)
    })

# 2. 健康检查接口
try:
    response = requests.get(f"{BASE_URL}/health", timeout=5)
    health_data = response.json()
    checks.append({
        '检查项': '健康检查接口',
        '状态': '✅ 通过' if health_data.get('status') == 'healthy' else '❌ 失败',
        '详情': f"模式: {health_data.get('mode', 'N/A')}"
    })

    CURRENT_MODE = health_data.get('mode', 'unknown')
    print(f"\n🔧 运行模式: {CURRENT_MODE.upper()}")

except Exception as e:
    checks.append({
        '检查项': '健康检查接口',
        '状态': '❌ 失败',
        '详情': str(e)
    })

# 3. API文档
try:
    response = requests.get(f"{BASE_URL}/docs", timeout=5)
    checks.append({
        '检查项': 'API文档可访问',
        '状态': '✅ 通过' if response.status_code == 200 else '❌ 失败',
        '详情': f"{BASE_URL}/docs"
    })
except Exception as e:
    checks.append({
        '检查项': 'API文档可访问',
        '状态': '❌ 失败',
        '详情': str(e)
    })

# 4. 环境检查
try:
    import edge_tts
    checks.append({
        '检查项': 'Edge TTS已安装',
        '状态': '✅ 通过',
        '详情': f"版本: {edge_tts.__version__}"
    })
except ImportError:
    checks.append({
        '检查项': 'Edge TTS已安装',
        '状态': '⚠️ 未安装',
        '详情': '语音功能可能受限'
    })

try:
    import websockets
    checks.append({
        '检查项': 'WebSocket库已安装',
        '状态': '✅ 通过',
        '详情': f"版本: {websockets.__version__}"
    })
except ImportError:
    checks.append({
        '检查项': 'WebSocket库已安装',
        '状态': '⚠️ 未安装',
        '详情': '实时推送测试将跳过'
    })

# 显示结果
df = pd.DataFrame(checks)
print("\n" + df.to_string(index=False))

TEST_RESULTS['health_check'] = checks

print("\n" + "=" * 70)
passed = sum(1 for c in checks if '✅' in c['状态'])
total = len(checks)
print(f"健康检查结果: {passed}/{total} 通过")
print("=" * 70)

🏥 系统健康检查
服务名称: 无障碍老年人导航后端
版本: 1.0.0

🔧 运行模式: PRODUCTION

          检查项   状态                         详情
        服务可达性 ✅ 通过                   状态码: 200
       健康检查接口 ✅ 通过             模式: production
     API文档可访问 ✅ 通过 http://localhost:8000/docs
  Edge TTS已安装 ✅ 通过                  版本: 7.2.7
WebSocket库已安装 ✅ 通过                   版本: 12.0

健康检查结果: 5/5 通过


In [32]:
import os
os.chdir('accessible-nav-backend')

print(f"切换后的目录: {os.getcwd()}")
print(f"\n现在的文件:")
print(os.listdir('.'))

切换后的目录: C:\Users\zy312\Desktop\高德竞赛代码\accessible-nav-backend\accessible-nav-backend

现在的文件:
['.env', '.env.example', 'app', 'audio_output', 'config', 'demo_test.ipynb', 'Dockerfile', 'main.py', 'QUICKSTART.md', 'quick_test.bat', 'quick_test.sh', 'README.md', 'requirements.txt', 'route_output', 'tests', 'yolov8n.pt', '__pycache__']


In [14]:
# 在Jupyter或Python中运行
import requests
from tqdm import tqdm

def download_yolo_model():
    url = "https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt"
    filename = "yolov8n.pt"

    print(f"正在下载: {url}")

    try:
        response = requests.get(url, stream=True, timeout=30)
        total_size = int(response.headers.get('content-length', 0))

        with open(filename, 'wb') as f, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True
        ) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    pbar.update(len(chunk))

        print(f"✅ 下载完成: {filename}")
        return True
    except Exception as e:
        print(f"❌ 下载失败: {e}")
        return False

# 执行下载
download_yolo_model()

正在下载: https://github.com/ultralytics/assets/releases/download/v0.0.0/yolov8n.pt


yolov8n.pt: 100%|██████████| 6.53M/6.53M [00:01<00:00, 4.68MB/s]

✅ 下载完成: yolov8n.pt


True

In [15]:
import os

# 检查文件是否存在
model_path = "yolov8n.pt"
if os.path.exists(model_path):
    file_size = os.path.getsize(model_path) / (1024 * 1024)  # MB
    print(f"✅ 模型文件存在")
    print(f"   路径: {model_path}")
    print(f"   大小: {file_size:.2f} MB")

    if file_size < 5:  # YOLOv8n应该大约6MB
        print("⚠️  文件大小异常，可能下载不完整")
    else:
        print("✅ 文件大小正常")
else:
    print("❌ 模型文件不存在")

✅ 模型文件存在
   路径: yolov8n.pt
   大小: 6.23 MB
✅ 文件大小正常


In [7]:
# 测试1: 简单问候
payload = {
    "userId": USER_ID,
    "sessionId": SESSION_ID,
    "text": "你好",
    "timestamp": int(time.time() * 1000)
}

response = requests.post(f"{BASE_URL}/v1/voice/text", json=payload)
result = response.json()

print("👤 用户:", payload["text"])
print("🤖 系统:", result["message"])
print("\n完整响应:")
print(json.dumps(result, indent=2, ensure_ascii=False))

👤 用户: 你好
🤖 系统: 我是您的导航助手，可以帮您规划路线。您想去哪里呢？

完整响应:
{
  "success": true,
  "message": "我是您的导航助手，可以帮您规划路线。您想去哪里呢？",
  "audioUrl": "/audio/mock_384ed293.wav",
  "navState": "asking",
  "data": {}
}
